In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import itertools as itr
import copy
from boundaries import WallBoundary, PressureBoundary, VelocityBoundary
from generators import WallGenerators
from simulation import Simulation
import pandas as pd
import random
from openpyxl import load_workbook

In [ ]:
yResolution = random.randint(20,50)
xResolution = random.randint(20,50)
ACspeed = 1

walls = WallBoundary(yResolution, xResolution)
walls.borderWall()
conditions = walls.generateRoom()
walls.generateIndex()
walls.generateACDirections()
walls.indexPossibleACPos()

bestPos = []
bestDirection = []
bestTemp = 0

In [ ]:
plt.imshow(walls.invertedBoundary, cmap="hot", interpolation="nearest")
plt.scatter([i[1] for i in walls.possibleACIndex], [i[0] for i in walls.possibleACIndex])
print(walls.possibleACIndex)

In [ ]:
def sim(pos):
    for directionIndex, direction in enumerate(WallBoundary.unitVect):
        #print(f"Trying direction: {direction}...")
        pathOfFlow = [i+j for i, j in zip(pos, direction)]
        reverseFlow = [i-j for i, j in zip(pos, direction)]
        isNormal = (not walls.boundary[pathOfFlow[0], pathOfFlow[1]]) and (walls.boundary[reverseFlow[0], reverseFlow[1]])

        if (isNormal):
            velocityInlet = [VelocityBoundary(pos[0], pos[1], ACspeed, directionIndex)]
            initCondition = np.ones((yResolution, xResolution, Simulation.latticeSize)) / 9
            simulation = Simulation(
                yResolution, xResolution, initCondition, walls, velocityBoundaries=velocityInlet, pressureBoundaries=[]
            )
            simTime, stable = simulation.simulateUntilEquilibrium(equilibriumThreshold=2.5, limit = 500)
            results = simulation.getMomentum()
            #print("Done!")
            if not stable or simTime == 500:
                #print("It exploded :() or didn't equalize")
                return False, results
            else:
                return True, results

In [ ]:
learningRate = 0.5

In [ ]:
currentIndex = 0
worked = False
length = len(walls.possibleACIndex)
visitedAll = np.zeros(length)
while True:
    visitedAll[currentIndex] = 1
    if (sum(visitedAll) == length):
        break
    
    currentPos = walls.possibleACIndex[currentIndex]
    print(f"Pos: {currentPos}")
    print("Simulating at pos...")
    usable, currentResults = sim(currentPos)
    if (not usable):
        print("Not stable")
        currentIndex += 1
        
    else:
        print("Stable")
        forwardPos = walls.possibleACIndex[(currentIndex + 1) % length]
        forwardUsable, forwardResults = sim(forwardPos)
        forwardDif = forwardResults - currentResults

        backwardPos = walls.possibleACIndex[(currentIndex - 1) % length] 
        backwardUsable, backwardResults = sim(backwardPos)
        backwardDif = currentResults - backwardResults

        forwardMessage = (str(forwardDif) if forwardUsable else "Unusable") 
        print(f"Forward Difference: {forwardMessage}")

        backwardMessage = (str(backwardDif) if backwardUsable else "Unusable") 
        print(f"Backward Difference: {backwardMessage}")

        if ((forwardDif < 0 and forwardUsable) and (backwardDif > 0 and backwardUsable)):
            print("Reached Equil.")
            worked = True
            break
        else:
            gradient = (forwardDif * forwardUsable) + (backwardDif * backwardUsable)
            print(f"Gradient: {gradient}")
            indexMove = learningRate * gradient
            if (indexMove < 1):
                print("Gradient too small. Pushing...")
                indexMove = np.sign(indexMove)
            
            currentIndex = (currentIndex + int(indexMove)) % length
    
    print("------------")

if (not worked):
    print("Didn't work.")
else:   
    print(f"Worked! {walls.possibleACIndex[currentIndex]}")
